# Toy Network

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx

from utils.layout import Layout
from utils.spheres import spheres, spherical_graph

ModuleNotFoundError: No module named 'aliases'

In [2]:
# SETTINGS

# scene parameters
dim = 3
samples = 50
start_radius = 0.2
end_radius = 1.1
step_size = 0.05
radii = np.arange(start_radius, end_radius+step_size, step_size)

# point cloud parameters
dim = 3
rng = np.random.default_rng(1)
n1 = 10
n2 = 5
perturb_amount = (0.25, 0.25, 0.5)
scale = 0.4
center = (0.5, 1, 0)

# colors and opacities
node_color = (102, 232, 188, 255) # turquoise
big_hole_hue = (248, 177, 0)      # yellow
little_hole_hue = (199, 56, 232)  # violet
base_opacity = 0
birth_opacity = 255
death_opacity = 50
wireframe_node_color = (0, 0, 0, 0)
wireframe_edge_color = (128, 128, 128, 20)

# holes thresholds
big_hole_birth = 0.5
big_hole_death = 1
little_hole_birth = 0.3
little_hole_death = 0.4

# persistence diagram parameters
figsize = (10, 5.625)
resolution = 101
dpi = 300
title = "Persistence Diagram"
xlabel = "time of birth"
ylabel = "time of death"
diag_color = "grey"

# environment variables
folder = "VR/toy/"
prefix = "toy_"

In [4]:
# UTILITY FUNCTIONS

def update_opacities(big_hole: int, little_hole: int) -> None:
    """Updates opacities of the hole edges."""
    layout.edge_colors = np.concatenate((
        np.repeat([big_hole_hue + (big_hole,)], repeats=n1, axis=0),
        np.repeat([little_hole_hue + (little_hole,)], repeats=n2, axis=0),
        np.repeat([wireframe_edge_color], repeats=G.number_of_edges()-PC.number_of_edges(), axis=0),
    ))

def update_positions(radius: float) -> None:
    """Updates positions of wireframe nodes to given radius."""
    _, pos = spheres(PC, PC_pos, radius=radius, samples=samples, dim=dim)
    layout.pos = pos

def draw_hole(radius: float, birth: float, death: float, color, start: float) -> None:
    """Draws the persistence diagram line for the given radius."""

    # solid line and marker over diagonal
    if np.allclose(radius, birth) or radius > birth:
        ax.vlines(birth, birth, min(radius, death), color=color)
        ax.plot(birth, min(radius, death), marker="o", color=color)

    # dashed vertical line under diagonal
    if np.allclose(radius, birth) or np.allclose(radius, death) or birth < radius < death:
        ax.vlines(birth, start, birth, color=color, linestyle="dashed")
    
    # dashed horizontal line at death
    if np.allclose(radius, death):
        ax.hlines(death, start, birth, color=color, linestyle="dashed")

In [107]:
# GENERATE POINT CLOUD

# network
C1 = spherical_graph(n1, dim=2)
C2 = spherical_graph(n2, dim=2)
nx.relabel_nodes(C2, lambda x: x+n1, copy=False)
PC = nx.compose(C1, C2)

# first circle positions
PC_pos = nx.circular_layout(C1, dim=dim)
for node in C1:
    PC_pos[node] = PC_pos[node] + perturb_amount * (0.5 - rng.random(dim))

# second circle positions flipped on y axis
pos = nx.circular_layout(C2, dim=dim, scale=scale, center=center)
for node in C2:
    PC_pos[node] = (pos[node] + perturb_amount * (0.5 - rng.random(dim)))[::-1]

# spheres with radius 0
G, pos = spheres(PC, PC_pos, radius=0, samples=samples, dim=dim)

In [108]:
# GENERATE LAYOUT

layout = Layout(G, pos)

# set node colors
layout.node_colors = np.concatenate((
    np.repeat([node_color], repeats=len(PC), axis=0),                      # point cloud nodes
    np.repeat([wireframe_node_color], repeats=(len(G) - len(PC)), axis=0), # sphere nodes
))

# set starting edge colors
update_opacities(base_opacity, base_opacity)

# save starting layout
layout.write(folder + prefix + "0")

In [109]:
# TEST DIFFERENT RADII
# LOWER samples VALUE FOR PREVIEWING

"""
update_positions(0.4)
update_opacities(birth_opacity, birth_opacity)
layout.preview(renderer="browser")
"""

'\nupdate_positions(0.4)\nupdate_opacities(birth_opacity, birth_opacity)\nlayout.preview(renderer="browser")\n'

In [110]:
# GENERATE SCENES

for i, radius in enumerate(radii):
    # update sphere positions and reset opacities
    update_positions(radius)
    update_opacities(base_opacity, base_opacity)

    # little hole
    if  np.allclose(radius, little_hole_birth) or little_hole_birth < radius < little_hole_death:
        update_opacities(base_opacity, birth_opacity)
    if np.allclose(radius, little_hole_death):
        update_opacities(base_opacity, death_opacity)

    # big hole
    if np.allclose(radius, big_hole_birth) or big_hole_birth < radius < big_hole_death:
        update_opacities(birth_opacity, base_opacity)
    if np.allclose(radius, big_hole_death):
        update_opacities(death_opacity, base_opacity)
    
    # export nodes and edges
    layout.write(folder + prefix + str(i+1))

In [5]:
# GENERATE PERSISTENCE DIAGRAMS

little_hole_color = np.array(little_hole_hue) / 256
big_hole_color = np.array(big_hole_hue) / 256

plot_start = radii[0] - step_size
plot_end = radii[-1] + step_size

for i, radius in enumerate(radii):

    # prepare figure
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title(title)
    ax.set_xticks(radii)
    ax.set_yticks(radii)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xlim(plot_start, plot_end)
    ax.set_ylim(plot_start, plot_end)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # diagonal
    line = np.linspace(plot_start, radius, resolution)
    ax.plot(line, line, color=diag_color)
    if radius < radii[-1]:
        ax.plot(radius, radius, marker="o", color=diag_color)

    # little hole
    draw_hole(radius, little_hole_birth, little_hole_death, little_hole_color, plot_start)
    
    # big hole
    draw_hole(radius, big_hole_birth, big_hole_death, big_hole_color, plot_start)
    
    # save plot and hide in notebook
    plt.savefig(folder + prefix + str(i+1) + "_diagram.jpg", dpi=dpi, bbox_inches="tight")
    plt.close()